<a href="https://colab.research.google.com/github/IG-Reagan/Mathematical-Modelling_Mixed-Integer-Linear-Program_Balanced-Syndicate-Group/blob/main/balanced_syndicate_group_MILP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Balanced Syndicate Group Optimisation - Excel OpenSolver Replication**

**Author:** Giwa Iziomo

**Objective:**
Maximise the average quantitative background (Q) within groups
while maintaining balance across gender and cultural background categories. i am replicating my initial solution with OpenSolver on MS Excel now with Python to increase versatility and scalability.

**Input:**
 - balanced_syndicate_dataset.csv

**Outputs:**
 - group_allocation.csv
 - group_statistics.csv

In [179]:
# Install PuLP if not already installed
try:
    import pulp
except ImportError:
    print("Installing PuLP library...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp", "-q"])
    print("PuLP installed successfully!\n")

In [180]:
# Import labraries

import pandas as pd
import numpy as np
from pulp import *
import warnings
warnings.filterwarnings('ignore')

In [181]:
# Mount Google Drive

# Mount Google Drive and load data
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Projects Portfolio/3.0. MSc in Business Analytics Research/Mathematical Modelling - Balanced Syndicate Group/balanced_syndicate_dataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [182]:
print("Dataset loaded successfully!")
print(f"Total students: {len(df)}")

Dataset loaded successfully!
Total students: 45


In [183]:
df.head()

,StudentID,Nationality,Cultural Background,Gender,Quantitative Background
0,101.0,Saudi Arabian,Middle East,Female,6.7
1,102.0,Saudi Arabian,Middle East,Male,4.0
2,103.0,Romanian,Eastern Europe,Female,5.9
3,104.0,British,British,Female,8.6
4,105.0,British,British,Male,5.0


In [184]:
# Data preprocessing

# Check Gender distribution
print("\nGender Distribution:")
print(df['Gender'].value_counts())


Gender Distribution:
Gender
Female    24
Male      21
Name: count, dtype: int64


In [185]:
# Check Cultural background distribution

print("\nCultural Background Distribution:")
print(df['Cultural Background'].value_counts())


Cultural Background Distribution:
Cultural Background
East Asia         20
British            9
Eastern Europe     6
Middle East        5
South Asia         3
Western Europe     2
Name: count, dtype: int64


In [186]:
# Check Nationality distribution

print("\nNationality Distribution:")
print(df['Nationality'].value_counts())


Nationality Distribution:
Nationality
Chinese          15
British           9
Saudi Arabian     3
Indian            3
Vietnamise        3
Russian           3
Romanian          1
Lithuanian        1
German            1
Spanish           1
Mongolian         1
Iranian           1
Turkish           1
Bulgarian         1
Japanese          1
Name: count, dtype: int64


In [187]:
# Check Quantitative background distribution

print("\nQuantitative Background Statistics:")
print(df['Quantitative Background'].describe())


Quantitative Background Statistics:
count    45.000000
mean      6.975556
std       1.538765
min       4.000000
25%       5.600000
50%       7.400000
75%       8.300000
max       9.200000
Name: Quantitative Background, dtype: float64


In [188]:
# Define Parameters
num_students = len(df)
num_groups = 9  # 45 students / 5 per group
group_size = 5


In [189]:
# Create student and group indices
students = df['StudentID'].tolist()
groups = list(range(1, num_groups + 1))

print(f"\nAllocating {num_students} students into {num_groups} groups of {group_size} members each")


Allocating 45 students into 9 groups of 5 members each


In [190]:
# Extract data for constraints
gender = df.set_index('StudentID')['Gender'].to_dict()
culture = df.set_index('StudentID')['Cultural Background'].to_dict()
nationality = df.set_index('StudentID')['Nationality'].to_dict()
quant_bg = df.set_index('StudentID')['Quantitative Background'].to_dict()

In [191]:
# Calculate target averages for balanced groups
avg_female_per_group = sum(1 for g in gender.values() if g == 'Female') / num_groups
avg_male_per_group = sum(1 for g in gender.values() if g == 'Male') / num_groups

print(f"\nTarget average females per group: {avg_female_per_group:.2f}")
print(f"Target average males per group: {avg_male_per_group:.2f}")


Target average females per group: 2.67
Target average males per group: 2.33


In [192]:
# Cultural background targets
culture_counts = df['Cultural Background'].value_counts()
culture_targets = {c: count / num_groups for c, count in culture_counts.items()}

print("\nTarget cultural background per group:")
for c, target in culture_targets.items():
    print(f"  {c}: {target:.2f}")


Target cultural background per group:
  East Asia: 2.22
  British: 1.00
  Eastern Europe: 0.67
  Middle East: 0.56
  South Asia: 0.33
  Western Europe: 0.22


In [193]:
# Create optimization model

model = LpProblem("Balanced_Syndicate_Groups", LpMaximize)

# Decision variables: x[s,g] = 1 if student s is assigned to group g
x = LpVariable.dicts("assign", [(s, g) for s in students for g in groups], cat='Binary')

In [194]:
# OBJECTIVE: Maximize average quantitative background while minimizing variance
# I'm using a weighted objective that balances both goals
avg_quant = sum(quant_bg.values()) / num_students

# Objective function: Maximize sum of quantitative backgrounds (proxy for balance)
model += lpSum([x[s, g] * quant_bg[s] for s in students for g in groups])

print("Objective function defined")

Objective function defined


In [195]:
# CONSTRAINTS

# 1. Each student must be in exactly one group
for s in students:
    model += lpSum([x[s, g] for g in groups]) == 1, f"Student_{s}_OneGroup"

print("Added constraint: Each student in exactly one group")

# 2. Each group must have exactly 5 members
for g in groups:
    model += lpSum([x[s, g] for s in students]) == group_size, f"Group_{g}_Size"

print("Added constraint: Each group has exactly 5 members")

# 3. Gender balance constraints (within acceptable range)
# Allow flexibility: target ± 1 for each gender
for g in groups:
    # Females in group g
    females_in_g = lpSum([x[s, g] for s in students if gender[s] == 'Female'])
    model += females_in_g >= avg_female_per_group - 1, f"Group_{g}_MinFemales"
    model += females_in_g <= avg_female_per_group + 1, f"Group_{g}_MaxFemales"

    # Males in group g (implicit from females constraint, but add for clarity)
    males_in_g = lpSum([x[s, g] for s in students if gender[s] == 'Male'])
    model += males_in_g >= avg_male_per_group - 1, f"Group_{g}_MinMales"
    model += males_in_g <= avg_male_per_group + 1, f"Group_{g}_MaxMales"

print("Added constraint: Gender balance across groups")

# 4. Cultural background balance constraints
# For each cultural background, try to distribute evenly
for c in culture_counts.index:
    target = culture_targets[c]
    for g in groups:
        students_with_culture = [s for s in students if culture[s] == c]
        if students_with_culture:
            culture_in_g = lpSum([x[s, g] for s in students_with_culture])
            # Allow some flexibility: target ± 1
            model += culture_in_g >= max(0, target - 1), f"Group_{g}_Min_{c.replace(' ', '_')}"
            model += culture_in_g <= target + 1.5, f"Group_{g}_Max_{c.replace(' ', '_')}"

print("Added constraint: Cultural background balance across groups")

# 5. British students constraint - at least 1 per group
british_students = [s for s in students if nationality[s] == 'British']

for g in groups:
    british_in_g = lpSum([x[s, g] for s in british_students])
    model += british_in_g >= 1, f"Group_{g}_MinBritish"

print("Added constraint: At least 1 British student per group")

# 6. Nationality pairing for international students
# Try to pair students with same nationality (excluding British)
non_british_nationalities = [n for n in df['Nationality'].unique() if n != 'British']

for nat in non_british_nationalities:
    students_with_nat = [s for s in students if nationality[s] == nat]
    if len(students_with_nat) >= 2:
        # Encourage pairing: if one student from this nationality is in a group,
        # try to have another one too
        for g in groups:
            nat_in_g = lpSum([x[s, g] for s in students_with_nat])
            # If any, encourage at least 2 (but don't require it strictly)
            # This is handled implicitly through the objective and other constraints

print("Added constraint: Nationality pairing preferences")

# 7. Quantitative background balance
# Ensure each group has similar average quantitative background
for g in groups:
    group_quant_sum = lpSum([x[s, g] * quant_bg[s] for s in students])
    # Target: each group should have average close to overall average
    # With 5 students per group, target sum = 5 * avg_quant
    target_sum = group_size * avg_quant
    # Allow variation of ±2 from target
    model += group_quant_sum >= target_sum - 2, f"Group_{g}_MinQuant"
    model += group_quant_sum <= target_sum + 2, f"Group_{g}_MaxQuant"

print("Added constraint: Quantitative background balance")

Added constraint: Each student in exactly one group
Added constraint: Each group has exactly 5 members
Added constraint: Gender balance across groups
Added constraint: Cultural background balance across groups
Added constraint: At least 1 British student per group
Added constraint: Nationality pairing preferences
Added constraint: Quantitative background balance


In [196]:
# OPTIMISATION MODEL

# Use CBC solver (default in PuLP)
solver = PULP_CBC_CMD(msg=1, timeLimit=300)  # 5 minute time limit
model.solve(solver)

# Check solution status
print("\n")
print("SOLUTION STATUS")
print("="*30)
print(f"Status: {LpStatus[model.status]}")
print(f"Objective Value: {value(model.objective):.2f}")

if model.status == 1:  # Optimal solution found

    # Extract solution
    allocation = {}
    for s in students:
        for g in groups:
            if value(x[s, g]) == 1:
                allocation[s] = g
                break

    # Create results dataframe
    results_df = df.copy()
    results_df['Assigned_Group'] = results_df['StudentID'].map(allocation)
    results_df = results_df.sort_values(['Assigned_Group', 'StudentID'])

    print("\n")
    print("GROUP ALLOCATION RESULTS")
    print("="*30)

    # Analyze each group
    group_stats = []

    for g in groups:
        group_students = results_df[results_df['Assigned_Group'] == g]

        stats = {
            'Group': f'G{g}',
            'Size': len(group_students),
            'Females': sum(group_students['Gender'] == 'Female'),
            'Males': sum(group_students['Gender'] == 'Male'),
            'Avg_Quant': group_students['Quantitative Background'].mean(),
        }

        # Count cultural backgrounds
        culture_dist = group_students['Cultural Background'].value_counts()
        for c in culture_counts.index:
            stats[f'{c}'] = culture_dist.get(c, 0)

        group_stats.append(stats)

    stats_df = pd.DataFrame(group_stats)

    print("\nGroup Summary:")
    print(stats_df.to_string(index=False))

    # Calculate variance metrics
    print("\n")
    print("BALANCE METRICS")
    print("="*30)

    avg_quants = stats_df['Avg_Quant'].values
    variance = np.var(avg_quants)
    std_dev = np.std(avg_quants)
    mean_quant = np.mean(avg_quants)
    cv = (std_dev / mean_quant) * 100 if mean_quant > 0 else 0

    print(f"\nQuantitative Background Variation:")
    print(f"  Variance: {variance:.4f}")
    print(f"  Standard Deviation: {std_dev:.4f}")
    print(f"  Mean: {mean_quant:.4f}")
    print(f"  Coefficient of Variation: {cv:.2f}%")

    print(f"\nGender Distribution:")
    print(f"  Female - Min: {stats_df['Females'].min()}, Max: {stats_df['Females'].max()}, Avg: {stats_df['Females'].mean():.2f}")
    print(f"  Male - Min: {stats_df['Males'].min()}, Max: {stats_df['Males'].max()}, Avg: {stats_df['Males'].mean():.2f}")

    # Detailed student allocation
    print("\n")
    print("DETAILED STUDENT ALLOCATION")
    print("="*30)

    display_cols = ['StudentID', 'Assigned_Group', 'Nationality',
                    'Cultural Background', 'Gender', 'Quantitative Background']
    print(results_df[display_cols].to_string(index=False))

    # Save results
    output_path = '/content/drive/My Drive/Projects Portfolio/3.0. MSc in Business Analytics Research/Mathematical Modelling - Balanced Syndicate Group/group_allocation.csv'
    results_df.to_csv(output_path, index=False)
    print(f"\nResults saved to: {output_path}")

    # Export group statistics
    stats_output_path = '/content/drive/My Drive/Projects Portfolio/3.0. MSc in Business Analytics Research/Mathematical Modelling - Balanced Syndicate Group/group_statistics.csv'
    stats_df.to_csv(stats_output_path, index=False)
    print(f"Group statistics saved to: {stats_output_path}")

else:
    print("\n Optimization did not find an optimal solution.")
    print("Consider relaxing some constraints or adjusting parameters.")



SOLUTION STATUS
Status: Optimal
Objective Value: 313.90


GROUP ALLOCATION RESULTS

Group Summary:
Group  Size  Females  Males  Avg_Quant  East Asia  British  Eastern Europe  Middle East  South Asia  Western Europe
   G1     5        2      3       7.04          2        1               1            1           0               0
   G2     5        3      2       7.04          2        1               1            1           0               0
   G3     5        3      2       7.12          3        1               0            0           0               1
   G4     5        2      3       6.96          2        1               0            1           1               0
   G5     5        2      3       6.78          2        1               1            1           0               0
   G6     5        3      2       7.04          2        1               2            0           0               0
   G7     5        3      2       6.76          3        1               0            0